In [4]:
import json

In [5]:
data_path = 'data/doc2dial/'
file_path = data_path  + 'doc2dial_dial_train.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [8]:
data['dial_data'].keys()

dict_keys(['dmv', 'ssa', 'va', 'studentaid'])

In [6]:
file_path = data_path + 'doc2dial_doc.json'
with open(file_path, 'r') as f:
    data = json.load(f)

In [4]:
len(data['doc_data']['ssa'].keys())

109

In [7]:
doc_list = list(data['doc_data']['ssa'].keys())

In [6]:
doc_list[0]

'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0'

In [7]:
data['doc_data']['ssa'][doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])

In [8]:
data['doc_data']['ssa'][doc_list[0]]['title']

'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1'

In [13]:
data['doc_data']['ssa'][doc_list[0]]['spans']

{'1': {'id_sp': '1',
  'tag': 'h2',
  'start_sp': 0,
  'end_sp': 61,
  'text_sp': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n',
  'title': 'Benefits Planner: Survivors | Planning For Your Survivors',
  'parent_titles': [],
  'id_sec': 't_0',
  'start_sec': 0,
  'end_sec': 61,
  'text_sec': '\n\nBenefits Planner: Survivors | Planning For Your Survivors \n'},
 '2': {'id_sp': '2',
  'tag': 'u',
  'start_sp': 61,
  'end_sp': 90,
  'text_sp': 'As you plan for the future , ',
  'title': 'Benefits Planner: Survivors | Planning For Your Survivors',
  'parent_titles': [],
  'id_sec': '1',
  'start_sec': 61,
  'text_sec': "As you plan for the future , you'll want to think about what your family would need if you should die now. Social Security can help your family if you have earned enough Social Security credits through your work. ",
  'end_sec': 274},
 '3': {'id_sp': '3',
  'tag': 'u',
  'start_sp': 90,
  'end_sp': 168,
  'text_sp': "you'll want to think about what your f

In [25]:
data['doc_data']['ssa'][doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])

In [8]:
def filter_and_write_to_json(data, keys, output_file) -> None:
    # Create a new dictionary with only the desired keys
    filtered_data = {key: data[key] for key in keys if key in data}

    # Write the filtered data to a JSON file
    with open(output_file, 'w') as file:
        json.dump(filtered_data, file)

In [9]:
keys_to_filter = ['title', 'doc_text']
filter_and_write_to_json(data['doc_data']['ssa'][doc_list[0]], keys_to_filter, 'data/doc1.json')

# VA

In [20]:
# data['doc_data']['va'].keys()
va_doc_list = list(data['doc_data']['va'].keys())

In [23]:
data['doc_data']['va'][va_doc_list[0]].keys()

dict_keys(['title', 'doc_id', 'domain', 'doc_text', 'spans', 'doc_html_ts', 'doc_html_raw'])

# Embedding

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/gtr-t5-base')

/home/ygong/miniconda3/envs/atlas-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)
print(embeddings)

[[-0.01774821  0.05172963  0.01236839 ... -0.00744141 -0.05631506
  -0.03496897]
 [-0.00638394  0.04372559  0.02519693 ... -0.00848611 -0.08687393
  -0.01826629]]


In [ ]:
def get_embeddings(batch, model):
    embeddings = model.encode(batch["text"])
    return {"embeddings": embeddings}

model = SentenceTransformer(
    "sentence-transformers/multi-qa-mpnet-base-dot-v1",
)

dataset = datasets.load_dataset("json", data_files="data.jsonl", split="train")
dataset = dataset.map(get_embeddings, batched=True, batch_size=32, fn_kwargs={"model": model})
dataset = dataset.with_format(
    type="numpy", columns=["embeddings"], output_all_columns=True,
)
dataset.add_faiss_index("embeddings")
dataset.save_faiss_index("embeddings", "index.faiss")

In [11]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='data/doc1.json',
    jq_schema='.doc_text',
)
document = loader.load()
print(f'documents:{len(document)}')

documents:1


In [109]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
# remove empty documents
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

documents:23


In [119]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/gtr-t5-base')
# embeddings = SentenceTransformer('sentence-transformers/gtr-t5-base')

# docsearch = Chroma.from_documents(split_documents, embeddings, persist_directory="data/vector_store")
# docsearch.persist()

# docsearch = Chroma(persist_directory="data/vector_store", embedding_function=embeddings)

db = FAISS.from_documents(split_documents, embeddings)


In [116]:
from langchain.vectorstores import Chroma
docsearch = Chroma.from_texts(split_documents, embeddings, metadatas=[{"source": str(i)} for i in range(len(split_documents))])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AttributeError: 'Document' object has no attribute 'replace'

In [60]:
query = "who need 40 credits?"
docs = db.similarity_search(query)

In [16]:
db.save_local("data/faiss_index")

In [18]:
new_db = FAISS.load_local("data/faiss_index", embeddings)

In [113]:
docs

[Document(page_content="needed to provide benefits for your survivors depends on your age when you die. No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit. But , the younger a person is , the fewer credits they must have for family members to receive survivors benefits. Benefits can be paid to your children and your spouse who is caring for the children even if you don't have the required number of credits. They can get benefits if you have credit for one and one - half", metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content='if you have credit for one and one - half years of work 6 credits in the three years just before your death.', metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="As you plan for the future , you'll want to think about what your family would need if you should die now. Social Security can help your family if you have e

In [1]:
paragraph = "Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles \"Crazy in Love\" and \"Baby Boy\"."

In [3]:
paragraph[207:]

'singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

# QA

In [32]:
from langchain import VectorDBQA
from langchain.chains import qa_with_sources
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = 'your_token'


In [64]:
# qa = qa_with_sources.from_chain_type(llm=qa_model, chain_type="map_reduce", vectorstore=db,return_source_documents=True)

qa_model = HuggingFaceHub(repo_id='google/flan-t5-base')
query = "Please answer the following question.\nHow need 40 credits?"
chain = load_qa_chain(llm=qa_model, chain_type="stuff")
chain.run(input_documents=docs, question=query, raw_response=True)


# result = qa({"query": "I'm looking for information regarding benefits planning, can you help me?"})
# print(result)

'No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit'

In [120]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
query = "Please answer the following question.\nHow need 40 credits?"
chain = load_qa_with_sources_chain(qa_model, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit'}

In [56]:
docs

[Document(page_content='Benefits Planner: Survivors | Planning For Your Survivors', metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="If you have a surviving divorced spouse , they could get the same benefits as your widow or widower provided that your marriage lasted 10 years or more. Benefits paid to a surviving divorced spouse won't affect the benefit amounts your other survivors will receive based on your earnings record. If your former spouse is caring for your child who is under age 16 or disabled and gets benefits on your record , they will not have to meet the length - of - marriage rule. The child must be your", metadata={'source': '/home/ygong/Attributed-QA/data/doc1.json', 'seq_num': 1}),
 Document(page_content="on your record , they can switch to their own retirement benefit as early as age 62. This assumes they're eligible for retirement benefits and their retirement rate is higher than their rate as a widow, widower, o

In [66]:
# read from a text file
with open('api.txt', 'r') as file:
    api = file.read()

In [68]:
os.environ["OPENAI_API_KEY"] = api

In [114]:
from langchain import OpenAI
query = "Please answer the following question.\nHow need 40 credits?"
openai_model = OpenAI(model_name="text-davinci-003", max_tokens=1024)
chain = load_qa_with_sources_chain(openai_model, chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

ImportError: Could not import openai python package. Please install it with `pip install openai`.